In [81]:
# Imports
import os
import pandas as pd
import numpy as np

# Matplotlib / Seaborn are optional for headless runs; try to import and fall back gracefully
try:
    import matplotlib.pyplot as plt
    import seaborn as sns
    PLOTTING_AVAILABLE = True
except Exception as e:
    plt = None
    sns = None
    PLOTTING_AVAILABLE = False

import joblib
from pathlib import Path

# Plot save control: when True, plots are saved to OUT_PLOTS_DIR instead of being embedded in the notebook
SAVE_PLOTS = True
OUT_PLOTS_DIR = Path('figures')
FIG_FORMAT = 'png'
if SAVE_PLOTS:
    OUT_PLOTS_DIR.mkdir(parents=True, exist_ok=True)

if PLOTTING_AVAILABLE:
    sns.set(style="whitegrid")
else:
    # provide a minimal placeholder to avoid NameError in the notebook if plotting is skipped
    class _Dummy:
        def __getattr__(self, name):
            def _noop(*args, **kwargs):
                return None
            return _noop
    sns = _Dummy()
    plt = None


In [82]:
# Load dataset (try local, else attempt to use kagglehub)
csv_path = Path('used_cars.csv')
if not csv_path.exists():
    try:
        import kagglehub
        dataset_dir = kagglehub.dataset_download('taeefnajib/used-car-price-prediction-dataset')
        candidate = Path(dataset_dir) / 'used_cars.csv'
        if candidate.exists():
            csv_path = candidate
    except Exception:
        pass

df = pd.read_csv(csv_path)
print('Loaded', csv_path)
df.shape

Loaded used_cars.csv


(4009, 12)

In [83]:
# Quick EDA: head, info, describe
display(df.head())
display(df.info())
display(df.describe(include='all').T)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,Ford,Utility Police Interceptor Base,2013,"51,000 mi.",E85 Flex Fuel,300.0HP 3.7L V6 Cylinder Engine Flex Fuel Capa...,6-Speed A/T,Black,Black,At least 1 accident or damage reported,Yes,"$10,300"
1,Hyundai,Palisade SEL,2021,"34,742 mi.",Gasoline,3.8L V6 24V GDI DOHC,8-Speed Automatic,Moonlight Cloud,Gray,At least 1 accident or damage reported,Yes,"$38,005"
2,Lexus,RX 350 RX 350,2022,"22,372 mi.",Gasoline,3.5 Liter DOHC,Automatic,Blue,Black,None reported,NaN,"$54,598"
3,INFINITI,Q50 Hybrid Sport,2015,"88,900 mi.",Hybrid,354.0HP 3.5L V6 Cylinder Engine Gas/Electric H...,7-Speed A/T,Black,Black,None reported,Yes,"$15,500"
4,Audi,Q3 45 S line Premium Plus,2021,"9,835 mi.",Gasoline,2.0L I4 16V GDI DOHC Turbo,8-Speed Automatic,Glacier White Metallic,Black,None reported,NaN,"$34,999"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4009 entries, 0 to 4008
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   brand         4009 non-null   object
 1   model         4009 non-null   object
 2   model_year    4009 non-null   int64 
 3   milage        4009 non-null   object
 4   fuel_type     3839 non-null   object
 5   engine        4009 non-null   object
 6   transmission  4009 non-null   object
 7   ext_col       4009 non-null   object
 8   int_col       4009 non-null   object
 9   accident      3896 non-null   object
 10  clean_title   3413 non-null   object
 11  price         4009 non-null   object
dtypes: int64(1), object(11)
memory usage: 376.0+ KB


None

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
brand,4009,57,Ford,386,NaN,NaN,NaN,NaN,NaN,NaN,NaN
model,4009,1898,M3 Base,30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
model_year,4009.0,NaN,NaN,NaN,2015.51559,6.104816,1974.0,2012.0,2017.0,2020.0,2024.0
milage,4009,2818,"110,000 mi.",16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fuel_type,3839,7,Gasoline,3309,NaN,NaN,NaN,NaN,NaN,NaN,NaN
engine,4009,1146,2.0L I4 16V GDI DOHC Turbo,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN
transmission,4009,62,A/T,1037,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ext_col,4009,319,Black,905,NaN,NaN,NaN,NaN,NaN,NaN,NaN
int_col,4009,156,Black,2025,NaN,NaN,NaN,NaN,NaN,NaN,NaN
accident,3896,2,None reported,2910,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
# Price distribution
if 'price' in df.columns:
    df['price_num'] = df['price'].astype(str).str.extract(r'(\d+\.?\d*)', expand=False)
    df['price_num'] = pd.to_numeric(df['price_num'], errors='coerce')
    price_vals = df['price_num'].dropna()
    # Only attempt plotting when plotting libs were imported successfully and plt is available
    if PLOTTING_AVAILABLE and plt is not None:
        if SAVE_PLOTS:
            plt.figure(figsize=(10,4))
            plt.subplot(1,2,1)
            sns.histplot(price_vals, bins=50, kde=True)
            plt.title('Price distribution')
            plt.subplot(1,2,2)
            sns.boxplot(x=price_vals)
            plt.title('Price boxplot')
            plt.tight_layout()
            plt.savefig(OUT_PLOTS_DIR / f'price_distribution.{FIG_FORMAT}', bbox_inches='tight')
            plt.close()
            print('Saved plot to', OUT_PLOTS_DIR / f'price_distribution.{FIG_FORMAT}')
        else:
            plt.figure(figsize=(10,4))
            plt.subplot(1,2,1)
            sns.histplot(price_vals, bins=50, kde=True)
            plt.title('Price distribution')
            plt.subplot(1,2,2)
            sns.boxplot(x=price_vals)
            plt.title('Price boxplot')
            plt.tight_layout()
    else:
        print('matplotlib/seaborn not available — skipping price distribution plots')
else:
    print('price column not found')

matplotlib/seaborn not available — skipping price distribution plots


In [85]:
# Mileage vs Price scatter (if available)
# Accept several possible column names: 'mileage_km', 'mileage', and the common typo 'milage'
mcols = [c for c in ['mileage_km', 'mileage', 'milage'] if c in df.columns]
if mcols:
    mcol = mcols[0]
    df['mileage_num'] = df[mcol].astype(str).str.extract(r'(\d+\.?\d*)', expand=False)
    df['mileage_num'] = pd.to_numeric(df['mileage_num'], errors='coerce')
    # Only attempt plotting when plotting libs were imported successfully and plt is available
    if PLOTTING_AVAILABLE and plt is not None:
        if SAVE_PLOTS:
            plt.figure(figsize=(6,4))
            sns.scatterplot(data=df, x='mileage_num', y='price_num')
            plt.xscale('log')
            plt.yscale('log')
            plt.xlabel('mileage (log)')
            plt.ylabel('price (log)')
            plt.title('Mileage vs Price (log-log)')
            plt.tight_layout()
            plt.savefig(OUT_PLOTS_DIR / f'mileage_vs_price.{FIG_FORMAT}', bbox_inches='tight')
            plt.close()
            print('Saved plot to', OUT_PLOTS_DIR / f'mileage_vs_price.{FIG_FORMAT}')
        else:
            plt.figure(figsize=(6,4))
            sns.scatterplot(data=df, x='mileage_num', y='price_num')
            plt.xscale('log')
            plt.yscale('log')
            plt.xlabel('mileage (log)')
            plt.ylabel('price (log)')
            plt.title('Mileage vs Price (log-log)')
            plt.tight_layout()
    else:
        print('matplotlib/seaborn not available — skipping mileage vs price plot')
else:
    print('mileage column not found')

matplotlib/seaborn not available — skipping mileage vs price plot


In [86]:
# Load trained pipeline and show feature rankings (importances or coefficients)
model_path = Path('car_price_pipeline.joblib')
if not model_path.exists():
    model_path = Path('ML assignment') / 'car_price_pipeline.joblib'
if not model_path.exists():
    print('Saved pipeline not found at', model_path)
else:
    import warnings
    try:
        from sklearn.exceptions import InconsistentVersionWarning
        warnings.filterwarnings('ignore', category=InconsistentVersionWarning)
    except Exception:
        pass
    pipeline = joblib.load(model_path)
    print('Loaded pipeline from', model_path)
    try:
        pre = pipeline.named_steps.get('preprocess') or pipeline.named_steps.get('preprocessor')
        if pre is None:
            raise KeyError("ColumnTransformer step not found (expected 'preprocess' or 'preprocessor')")
        num_cols = [t for t in pre.transformers_ if t[0] == 'num'][0][2]
        cat_transform = [t for t in pre.transformers_ if t[0] == 'cat'][0]
        cat_cols = cat_transform[2]
        enc_candidate = cat_transform[1]
        if hasattr(enc_candidate, 'named_steps') and 'onehot' in enc_candidate.named_steps:
            onehot = enc_candidate.named_steps['onehot']
        else:
            onehot = enc_candidate
        if hasattr(onehot, 'get_feature_names_out'):
            cat_names = onehot.get_feature_names_out(cat_cols)
        elif hasattr(onehot, 'get_feature_names'):
            cat_names = onehot.get_feature_names(cat_cols)
        else:
            cat_names = [f'{c}_OH' for c in cat_cols]
        feature_names = np.concatenate([num_cols, cat_names])
        model_step = pipeline.named_steps['model']
        if hasattr(model_step, 'feature_importances_'):
            scores = model_step.feature_importances_
            label = 'feature_importances'
        elif hasattr(model_step, 'coef_'):
            coef = model_step.coef_
            if getattr(coef, 'ndim', 1) > 1:
                coef = coef.ravel()
            scores = np.abs(coef)
            label = 'abs(coef_)'
        else:
            raise AttributeError('Model has neither feature_importances_ nor coef_')
        fi = pd.DataFrame({'feature': feature_names, 'importance': scores}).sort_values('importance', ascending=False).reset_index(drop=True)
        display(fi.head(30))
        print(f'Displayed top features using {label}.')
        if PLOTTING_AVAILABLE and plt is not None:
            if SAVE_PLOTS:
                plt.figure(figsize=(8,6))
                sns.barplot(data=fi.head(20), x='importance', y='feature')
                plt.title('Top 20 feature rankings')
                plt.tight_layout()
                plt.savefig(OUT_PLOTS_DIR / f'feature_importances.{FIG_FORMAT}', bbox_inches='tight')
                plt.close()
                print('Saved plot to', OUT_PLOTS_DIR / f'feature_importances.{FIG_FORMAT}')
            else:
                plt.figure(figsize=(8,6))
                sns.barplot(data=fi.head(20), x='importance', y='feature')
                plt.title('Top 20 feature rankings')
                plt.tight_layout()
        else:
            print('matplotlib/seaborn not available — skipping feature ranking plot')
    except Exception as e:
        print('Could not extract feature rankings:', e)

Loaded pipeline from car_price_pipeline.joblib


,feature,importance
0,car_age,0.348064
1,mileage_km,0.269824
2,hp,0.100990
3,engine_cc,0.100004
4,brand_freq,0.064712
5,model_freq,0.020412
6,fuel_Diesel,0.005685
7,transmission_7-Speed Automatic with Auto-Shift,0.005376
8,int_col_Nero Ade,0.003552
9,transmission_A/T,0.003255


Displayed top features using feature_importances.
matplotlib/seaborn not available — skipping feature ranking plot


In [87]:
# Sample predictions using the loaded pipeline (engineered features)
if 'pipeline' in globals():
    sample = df.sample(5, random_state=42).copy()
    actual = None
    if 'price_num' in sample.columns:
        actual = sample['price_num']
        sample = sample.drop(columns=['price_num'], errors='ignore')

    def preprocess_df(df_in: pd.DataFrame) -> pd.DataFrame:
        col_map_actual = {'model_year': 'year', 'milage': 'mileage_km', 'mileage': 'mileage_km', 'fuel_type': 'fuel'}
        dfx = df_in.rename(columns=col_map_actual)
        if 'price' in dfx.columns:
            dfx['price'] = dfx['price'].astype(str).str.extract(r'(\d+\.?\d*)', expand=False)
            dfx['price'] = pd.to_numeric(dfx['price'], errors='coerce')
            dfx = dfx.dropna(subset=['price'])
        if 'mileage_km' in dfx.columns:
            dfx['mileage_km'] = dfx['mileage_km'].astype(str).str.extract(r'(\d+\.?\d*)', expand=False)
            dfx['mileage_km'] = pd.to_numeric(dfx['mileage_km'], errors='coerce')
        if 'engine' in dfx.columns:
            dfx['engine_cc'] = dfx['engine'].astype(str).str.extract(r'(\d+\.?\d*)', expand=False)
            dfx['engine_cc'] = pd.to_numeric(dfx['engine_cc'], errors='coerce')
            hp = dfx['engine'].astype(str).str.extract(r'(\d+\.?\d*)\s*HP', expand=False)
            hp = hp.fillna(dfx['engine'].astype(str).str.extract(r'^(\d+\.?\d*)', expand=False))
            dfx['hp'] = pd.to_numeric(hp, errors='coerce')
        if 'year' in dfx.columns:
            dfx['year'] = pd.to_numeric(dfx['year'], errors='coerce')
            dfx['car_age'] = pd.Timestamp.now().year - dfx['year']
        if 'brand' in dfx.columns and 'brand' in df.columns:
            dfx['brand_freq'] = dfx['brand'].map(df['brand'].value_counts()).fillna(0)
        if 'model' in dfx.columns and 'model' in df.columns:
            dfx['model_freq'] = dfx['model'].map(df['model'].value_counts()).fillna(0)
        if 'accident' in dfx.columns:
            dfx['accident_flag'] = (~dfx['accident'].astype(str).str.contains('none', case=False, na=False)).astype(int)
        if 'clean_title' in dfx.columns:
            dfx['clean_title_flag'] = dfx['clean_title'].astype(str).str.lower().eq('yes').astype(int)
        drop_cols = [c for c in ['name','year','brand','model','engine'] if c in dfx.columns]
        dfx = dfx.drop(columns=drop_cols, errors='ignore')
        return dfx

    X_sample = preprocess_df(sample.copy())
    try:
        raw_preds = pipeline.predict(X_sample)
        if np.nanmax(raw_preds) < 20:
            preds = np.expm1(raw_preds)
        else:
            preds = raw_preds
        out = sample.copy()
        out['predicted_price'] = preds
        if actual is not None:
            out['actual_price'] = actual.values
        display(out.head())
    except Exception as e:
        print('Prediction failed:', e)
else:
    print('Pipeline not loaded; cannot run sample predictions')

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price,mileage_num,predicted_price,actual_price
2580,Lexus,IS 300 Base,2018,"50,992 mi.",Gasoline,260.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,A/T,White,Gray,At least 1 accident or damage reported,Yes,"$28,000",50,27.887094,28
3660,Chevrolet,Impala Base,2004,"64,500 mi.",Gasoline,180.0HP 3.4L V6 Cylinder Engine Gasoline Fuel,A/T,Beige,Beige,None reported,Yes,"$5,900",64,9.740833,5
897,RAM,2500 SLT,2017,"86,000 mi.",Diesel,350.0HP 6.7L Straight 6 Cylinder Engine Diesel...,6-Speed A/T,Gray,Gray,At least 1 accident or damage reported,Yes,"$41,000",86,37.774170,41
2091,Mercedes-Benz,SL-Class SL 550,2013,"24,933 mi.",Gasoline,429.0HP 4.6L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Silver,Red,At least 1 accident or damage reported,Yes,"$40,250",24,40.713865,40
1044,Ford,Shelby GT350R Base,2018,"18,500 mi.",Gasoline,526.0HP 5.2L 8 Cylinder Engine Gasoline Fuel,M/T,Blue,Black,At least 1 accident or damage reported,Yes,"$77,999",18,66.185782,77


## Conclusion
The RandomForest pipeline is the best performing model from our experiments and is saved as a joblib pipeline for reproducible inference. For a school submission, include this notebook, the `train_final.py` and `predict_final.py` scripts, the saved pipeline, and the brief report `REPORT_FINAL.md`.

Next steps: feature engineering (brand/model extraction), target encoding for high-cardinality fields, and trying LightGBM/XGBoost for potential further gains.